In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import colors as mcolors
colors = mcolors.CSS4_COLORS
import seaborn as sns
import requests

import cufflinks as cf
cf.set_config_file(offline=True, world_readable=False, theme='ggplot')

import sys, datetime, time, pickle
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 7.0)

In [ ]:
def set_y_axis_thousands_delim(ax):
    """Accept a Matplotlib Axes object, and formats the y-ticks to be integers with comma separators, e.g.
    ax = df.plot()
    set_y_axis_thousands_delim(ax)
    """
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda y, p: format(int(y), ',')))


# Global data
**Data downloaded from here:**
https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide

In [ ]:
# # V1: This was only valid 3/16
# df = pd.read_excel('~/Downloads/COVID-19-geographic-disbtribution-worldwide-2020-03-16.xls')
# col_mapping = {'DateRep':'Date', 'CountryExp':'Country', "NewConfCases":"Cases", "NewDeaths":"Deaths"}

# # V2: There was some inconsistency in file format between xls and xlsx for a few days
# today = datetime.date.today()
# # today = datetime.date(2020,3,19)
# try:
#     df = pd.read_excel('~/Downloads/COVID-19-geographic-disbtribution-worldwide-%s.xlsx'%today.strftime('%Y-%m-%d'))
# except IOError:
#     df = pd.read_excel('~/Downloads/COVID-19-geographic-disbtribution-worldwide-%s.xls'%today.strftime('%Y-%m-%d'))
# col_mapping = {'DateRep':'Date', 'Countries and territories':'Country'}

In [ ]:
# V3: ECDC standard request process
from urllib2 import HTTPError

testdate = datetime.date.today()
url = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{:}.xlsx"

try:
    df = pd.read_excel(url.format(testdate.strftime('%Y-%m-%d')))
    print("Successfully loaded data for today, %s"%testdate)
except HTTPError:
    testdate -= datetime.timedelta(days=1)
    df = pd.read_excel(url.format(testdate.strftime('%Y-%m-%d')))
    print("Today's data not yet available, using data for %s"%testdate)

col_mapping = {'DateRep':'Date', 'Countries and territories':'Country'}

In [ ]:
df.columns = df.columns.map(lambda x: col_mapping.get(x,x))
df = df.set_index(['Country', 'Date'], drop=True).sort_index()
df = df[['Cases', 'Deaths']]
df.head()

In [ ]:
global_cases = df['Cases'].groupby('Date').sum().cumsum()
global_deaths = df['Deaths'].groupby('Date').sum().cumsum()

china_cases = df.loc[('China',slice(None)),:]['Cases'].reset_index(level=0, drop=True).cumsum()
china_deaths = df.loc[('China',slice(None)),:]['Deaths'].reset_index(level=0, drop=True).cumsum()

In [ ]:
def plot_growth(s, a=5, ax=None, ylims=(0,2.0)):
    realized_color='indianred'
    mean_color = 'cornflowerblue'

    a = 5
    g = s.diff() / s.diff().shift(1)
    g.plot(color=colors[realized_color], ax=ax)
    ax.axhline(1.0, color='k', linewidth=2)
    ax.axhline(1.25,color='k', linewidth=1, linestyle=':')
    ax.set_ylim(ylims)
    m = g.rolling(a).mean()
    m.plot(color=mean_color, linestyle='--', ax=ax)

    err = g.rolling(a).apply(lambda x: np.std([np.random.choice(x, size=a, replace=True) for i in range(100)]))

    ax.fill_between(m.index, m-err, m+err, color=mean_color, alpha=0.2)
    ax.set_ylabel('Growth Rate')

# Global Caseload

In [ ]:
print("Current global cases: {:,.0f}".format(global_cases.iloc[-1]))
print("Changes in global cases in last 3 days:")
global_cases.diff().tail(3)

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True, figsize=(15,10))

global_cases.plot(label='Global', ax=ax[0])
china_cases.plot(label='China', ax=ax[0])
set_y_axis_thousands_delim(ax[0])
ax[0].legend()
ax[0].set_title('Total Global Cases')
ax[0].set_ylabel('Total cases')

plot_growth(global_cases, ax=ax[1])

## Global Deaths

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True, figsize=(15,10))

global_deaths.plot(label='Global', ax=ax[0])
china_deaths.plot(label='China', ax=ax[0])
set_y_axis_thousands_delim(ax[0])
ax[0].legend()

plot_growth(global_deaths, ax=ax[1])

# World excluding China (Rest of World or RoW)

## RoW cases

In [ ]:
row_cases = (global_cases - china_cases)

fig, ax = plt.subplots(2,1, sharex=True, figsize=(15,10))

row_cases.plot(ax=ax[0])
set_y_axis_thousands_delim(ax[0])
ax[0].set_ylabel('Total cases, RoW')
ax2 = row_cases.diff().plot(ax=ax[0],secondary_y=True, linestyle='--')
set_y_axis_thousands_delim(ax2)
ax2.set_ylabel('New case rate')

plot_growth(row_cases, ax=ax[1])

## RoW Deaths

In [ ]:
row_deaths = global_deaths - china_deaths

fig, ax = plt.subplots(2,1, sharex=True, figsize=(15,10))

row_deaths.plot(ax=ax[0])
set_y_axis_thousands_delim(ax[0])
ax[0].set_ylabel('Total Deaths, RoW')
ax2 = row_deaths.diff().plot(ax=ax[0],secondary_y=True, linestyle='--')
set_y_axis_thousands_delim(ax2)
ax2.set_ylabel('New Death rate')

plot_growth(row_deaths, ax=ax[1])

# Country-specific stats

In [ ]:
by_country = df.groupby(level=0).sum()
by_country.loc['Global',:] = by_country.sum()

n_cutoff = 10

for f in ['Cases', 'Deaths']:
    print(f)
    new_cases = pd.concat([by_country[f], by_country[f] / by_country.loc['Global', f] * 100], axis=1, keys=[f, 'Portion'])
    display(new_cases.sort_values(by=f, ascending=False).iloc[:n_cutoff].style.format({f:'{:,.0f}', 'Portion':'{:.2f}%'}))

    
print("Mortality")    
death_cutoff = 10
mortality = by_country[by_country['Deaths'] > death_cutoff]
mortality['Mortality Rate'] = mortality['Deaths'] / mortality['Cases'] * 100
mortality = mortality[['Mortality Rate', 'Deaths', 'Cases']]
mortality = mortality.sort_values(by='Mortality Rate', ascending=False)
mortality.style.format({'Mortality Rate':'{:.1f}%', 'Deaths':'{:,.0f}', 'Cases':'{:,.0f}'})

In [ ]:
df.loc[('Austria',slice(None)),:].tail()

In [ ]:
field = 'Cases'

countries_for_detail = df.groupby(level=0).sum().sort_values(by=field, ascending=False).index[:n_cutoff]

# countries_for_detail = ['Indonesia']

for c in countries_for_detail:
    s = df.loc[(c, slice(None)),:].reset_index(level=0, drop=True)
    s_cases = s[field]
    sustained_start_date = s_cases[(s_cases.diff() * s_cases.diff().shift(-1))>0].index[0]

    fig, ax = plt.subplots(2,1, sharex=True, figsize=(15,10))

    s_cases.cumsum()[sustained_start_date:].plot(ax=ax[0])
    set_y_axis_thousands_delim(ax[0])
    ax[0].set_ylabel('Total number of '+field)
    ax[0].set_title('{:}:  {:,.0f} cases,  {:,.0f} deaths,  sustained reporting started {:}  ({:.0f} days)'.format(c,
                    s.sum()['Cases'], s.sum()['Deaths'], sustained_start_date.strftime('%m-%d'), (datetime.datetime.now() - sustained_start_date).days))
    ax2 = s_cases[sustained_start_date:].plot(linestyle='--',ax=ax[0], secondary_y=True)
    ax2.set_ylabel('New Cases')

    plot_growth(s_cases[sustained_start_date:].cumsum(), ax=ax[1])
    plt.show()
    plt.close()

# US Data

In [ ]:
tmp = requests.get('https://covidtracking.com/api/states/daily')
us_data = pd.DataFrame(json.loads(tmp.content))

field_mapping = {'date':'Date','death':'Deaths','positive':'Cases','state':'State', 'total':'Total Tested'}
us_data.columns = us_data.columns.map(lambda x: field_mapping.get(x,x))

us_data['Date'] = pd.to_datetime(us_data['Date'],format='%Y%m%d')
us_data = us_data.set_index(['State','Date'], drop=True).sort_index()

In [ ]:
us_data.groupby('Date').sum()

In [ ]:
# Compare WHO and CDC numbers. US CDC numbers are released later, which is likely the explanation for the difference. 
us_total = us_data.groupby('Date').sum()

s_cases = df.loc[('United_States_of_America', slice(None)),:].reset_index(level=0, drop=True)

f = 'Cases'
us_case_totals = pd.concat([us_total[f], s_cases[f].cumsum()], axis=1).dropna()
us_case_totals.columns = ['US CDC', 'WHO/EUCDC']
us_case_totals.iplot()

In [ ]:
(us_total[f].diff() / us_total[f].diff().shift(1)).iplot(dimensions=(600,400))

In [ ]:
testing = us_total.diff()
ax = testing[['Cases','Total Tested']].plot()
set_y_axis_thousands_delim(ax)
handles, labels = ax.get_legend_handles_labels()
handleDict = dict(zip(labels,handles))

ax2 = (testing['Cases']/testing['Total Tested']).plot(label='Portion positive on day', secondary_y=True, linestyle='--')
handles, labels = ax2.get_legend_handles_labels()
handleDict.update(zip(labels,handles))

ax.legend(handleDict.values(), handleDict.keys(), loc='upper left')

plt.title("Portion of day's tests positive")

In [ ]:
biggest_states = us_data.groupby('State').last()[['Cases','Deaths']].sort_values('Cases', ascending=False).iloc[0:n_cutoff,:]
biggest_states

In [ ]:
states_for_detail = biggest_states.index

for c in states_for_detail:
    s = us_data.loc[(c, slice(None)),:].reset_index(level=0, drop=True)
    s_cases = s['Cases']

    fig, ax = plt.subplots(2,1, sharex=True, figsize=(15,10))

    s_cases.plot(ax=ax[0])
    set_y_axis_thousands_delim(ax[0])
    ax[0].set_ylabel('Total number of cases')
    ax[0].set_title('{:}:  {:,.0f} cases,  {:,.0f} deaths'.format(c, s['Cases'][-1], s['Deaths'][-1]))
    ax2 = s_cases.diff().plot(linestyle='--',ax=ax[0], secondary_y=True)
    ax2.set_ylabel('New Cases')

    plot_growth(s_cases.fillna(0.0), ax=ax[1], ylims=(0,3.0))
    plt.show()
    plt.close()